In [1]:
import requests
from bs4 import BeautifulSoup as bs
import re
r=requests.get("https://en.wikipedia.org/wiki/Toy_Story_3")
soup=bs(r.content)
#print(soup.prettify())

In [36]:
import re
dict={}

def get_value(row):
    value=[]
    if row.find("li"):
        for li in row.find_all("li"):
            value.append(li.get_text(" ", strip=True).replace('\xa0',' '))
        return(value)
    else:
        return(row.td.get_text(" ", strip=True).replace('\xa0',' '))

table= soup.find("table", attrs={"class": "infobox vevent"}).find_all("tr")
for index,row in enumerate(table):
    if index==0:
        dict['title']=row.th.get_text(" ", strip=True)
    elif index>1:
        dict[row.th.get_text(" ", strip=True)]= get_value(row)


    

In [2]:
import re
def clean_tags(soup):
    for tag in soup.find_all("sup"):
        tag.decompose()
    for tag in soup.find_all("span"):
        if tag.find(string=re.compile("million")) or tag.find(string=re.compile("billion")) :
            continue
        tag.decompose()
        
def get_value(row):
        value=[]
        if row.find("li"):
            for li in row.find_all("li"):
                value.append(li.get_text(" ", strip=True).replace('\xa0',' '))
            return(value)
        elif row.find("br"):
            if row.find("a") and row.th.get_text()!="Running time" and row.th.get_text()!="Budget" and row.th.get_text()!="Box office":
                for a in row.find_all("a"):
                    value.append(a.get_text(" ", strip=True).replace('\xa0',' '))
                return(value)
            else:
                return [text for text in row.td.stripped_strings]
        else:
            return(row.td.get_text(" ", strip=True).replace('\xa0',' '))

def get_info(movie_url):
    r=requests.get(movie_url)
    soup=bs(r.content)
    clean_tags(soup)
    dict={}
    table= soup.find("table", attrs={"class": "infobox vevent"}).find_all("tr")
    for index,row in enumerate(table):
        if index==0:
            dict['title']=row.th.get_text(" ", strip=True)
        elif index>1:
            header=row.find("th")
            if header:
                dict[row.th.get_text(" ", strip=True)]= get_value(row)
    
    return dict



In [38]:
get_info("https://en.wikipedia.org/wiki/Tron")

{'title': 'Tron',
 'Directed by': 'Steven Lisberger',
 'Screenplay by': 'Steven Lisberger',
 'Story by': ['Steven Lisberger', 'Bonnie MacBird'],
 'Produced by': 'Donald Kushner',
 'Starring': ['Jeff Bridges',
  'Bruce Boxleitner',
  'David Warner',
  'Cindy Morgan',
  'Barnard Hughes'],
 'Cinematography': 'Bruce Logan',
 'Edited by': 'Jeff Gourson',
 'Music by': 'Wendy Carlos',
 'Production companies': ['Walt Disney Productions',
  'Lisberger-Kushner Productions'],
 'Distributed by': 'Buena Vista Distribution',
 'Release date': ['July 9, 1982'],
 'Running time': '96 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Budget': 'US$ 17 million',
 'Box office': 'US$50 million'}

In [3]:
r=requests.get("https://en.wikipedia.org/wiki/List_of_Walt_Disney_Pictures_films")
soup=bs(r.content)
movies_info_list=[]
url="https://en.wikipedia.org"
movies= soup.select(".wikitable.sortable i a")
for index,movie in enumerate(movies):
    try:
        if index>0:
            full_url=url+movie["href"]
            movies_info_list.append(get_info(full_url))
    except Exception as e:
        print(movie.get_text())
        print(e)
        
        

Zorro the Avenger
'NoneType' object has no attribute 'get_text'
The Sign of Zorro
'NoneType' object has no attribute 'get_text'
True-Life Adventures
'NoneType' object has no attribute 'find_all'
The London Connection
'NoneType' object has no attribute 'get_text'
Better Nate Than Never
'NoneType' object has no attribute 'find_all'


In [4]:
import json                                         #saving python into json

def save_data(title, data):
    with open(title,'w', encoding="utf-8") as f:
        json.dump(data,f, ensure_ascii=False, indent=2)

In [5]:
def load_data(title):                                   #json to python
    with open(title, encoding="utf-8") as f:
        return json.load(f)

In [6]:
save_data("disney_data.json",movies_info_list)

In [7]:
moives_info_list=load_data("disney_data.json")

In [8]:
def minutes_to_integer(running_time):
    if running_time=="N/A":
        return None
    
    if isinstance(running_time,  __builtins__.list):
        return int(running_time[0].split(" ")[0])
    else:
        return int(running_time.split(" ")[0])


for index,movies in enumerate(movies_info_list):
    try:
        movies['Running time (int)']=minutes_to_integer(movies.get('Running time',"N/A"))
    except Exception as e:
        print(index,e)

In [16]:
import re
number=r"\d+(,\d{3})*(\.\d+)*"

def converter(money,value):
    val_dict={"million": 1000000,"billion": 1000000000, "crore": 10000000}
    if "million" in money:
        return value*val_dict["million"]
    elif "billion" in money:
        return value*val_dict["billion"]
    elif "crore" in money:
        return value*val_dict["crore"]
    else:
        return int(value)

    
def suffix(money):
    for element in money:
        c_list=["million","billion","crore"]
        for suffix in c_list:
            if suffix in element:
                return element

            
def money_conversion(money):
    if money=="N/A":
            return None  
    elif isinstance(money,  __builtins__.list):
        money_from_list= suffix(money)
        Lvalue=float(re.search(number, money_from_list.replace(",","")).group())
        return converter(money_from_list,Lvalue)
    else:
        Lvalue= float(re.search(number, money.replace(",","")).group())
        return converter(money,Lvalue)
 
   
        

In [17]:
for index,movies in enumerate(movies_info_list):
    try:
        movies['Budget (int)']=money_conversion(movies.get('Budget',"N/A"))
    except Exception as e:
        print(index,e)
    try:
        movies['Box office (int)']=money_conversion(movies.get('Box office',"N/A"))
    except Exception as e:
        print(index,e)

154 'NoneType' object has no attribute 'group'


In [20]:
from datetime import datetime 
def date_conversion(date):
    if date=="N/A":
        return None  
    elif isinstance(date, __builtins__.list):
        Rdate= date[0].split(" (")[0]
    else:
        Rdate=date
    
    frmts=["%B %d, %Y", "%d %B, %Y"]
    for frmt in frmts:
        try:
            return datetime.strptime(Rdate, frmt)
        except:
            pass

        

In [21]:
for index,movies in enumerate(movies_info_list):
    try:
        movies['Release date (datetime)']=date_conversion(movies.get('Release date',"N/A"))
    except Exception as e:
        print(index,e)

In [22]:
movies_info_list[30]

{'title': 'Davy Crockett and the River Pirates',
 'Directed by': 'Norman Foster',
 'Written by': ['Tom Blackburn'],
 'Produced by': 'Bill Walsh',
 'Starring': ['Fess Parker', 'Buddy Ebsen', 'Jeff York'],
 'Edited by': 'Stanley Johnson',
 'Music by': ['George Bruns', 'Edward H. Plumb'],
 'Production company': ['Walt Disney Productions'],
 'Distributed by': 'Buena Vista Film Distribution Co., Inc.',
 'Release date': 'July 18, 1956',
 'Running time': '81 minutes',
 'Country': 'United States',
 'Language': 'English',
 'Running time (int)': 81,
 'Budget (int)': None,
 'Box office (int)': None,
 'Release date (datetime)': datetime.datetime(1956, 7, 18, 0, 0)}

In [23]:
import pickle

def save_data_pickle(name, data):
    with open(name, "wb") as f:
        pickle.dump(data, f)

In [24]:
import pickle

def load_data_pickle(name):
    with open(name, "rb") as f:
        return pickle.load(f)

In [25]:
save_data_pickle("disney_data_clean.pickle", movies_info_list)

In [26]:
movies_info_list=load_data_pickle("disney_data_clean.pickle")

In [27]:
import requests
import urllib

def get_omdb_info(title):
    base_url="http://www.omdbapi.com/?"
    parameters={"apikey": "30bfc907", "t": title}
    param_encoded=urllib.parse.urlencode(parameters)
    full_url=base_url + param_encoded
    return requests.get(full_url).json()

In [28]:
get_omdb_info("into the woods")

{'Title': 'Into the Woods',
 'Year': '2014',
 'Rated': 'PG',
 'Released': '25 Dec 2014',
 'Runtime': '125 min',
 'Genre': 'Adventure, Comedy, Drama, Fantasy, Musical',
 'Director': 'Rob Marshall',
 'Writer': 'James Lapine (screenplay by), James Lapine (based on the musical by)',
 'Actors': 'Anna Kendrick, Daniel Huttlestone, James Corden, Emily Blunt',
 'Plot': 'A witch tasks a childless baker and his wife with procuring magical items from classic fairy tales to reverse the curse put on their family tree.',
 'Language': 'English',
 'Country': 'USA',
 'Awards': 'Nominated for 3 Oscars. Another 10 wins & 71 nominations.',
 'Poster': 'https://m.media-amazon.com/images/M/MV5BMTY4MzQ4OTY3NF5BMl5BanBnXkFtZTgwNjM5MDI3MjE@._V1_SX300.jpg',
 'Ratings': [{'Source': 'Internet Movie Database', 'Value': '5.9/10'},
  {'Source': 'Rotten Tomatoes', 'Value': '71%'},
  {'Source': 'Metacritic', 'Value': '69/100'}],
 'Metascore': '69',
 'imdbRating': '5.9',
 'imdbVotes': '134,673',
 'imdbID': 'tt2180411',


In [41]:
def get_rottontomatoes(omdb_dict):
    for rt in omdb_dict.get("Ratings", []):
        if rt["Source"]=="Rotten Tomatoes":
            return rt["Value"]
        

In [42]:
for movies in movies_info_list:
    try:
        omdb_info= get_omdb_info(movies.get('title'))
        movies["Metascore"]= omdb_info.get("Metascore", None)
        movies["imdbRating"]= omdb_info.get("imdbRating", None)
        movies["Rotton Tomatoes"]= get_rottontomatoes(omdb_info) 
    except Exception as e:
        print(index,e)

In [43]:
movies_info_list

[{'title': 'Snow White and the Seven Dwarfs',
  'Directed by': ['David Hand (supervising)',
   'William Cottrell',
   'Wilfred Jackson',
   'Larry Morey',
   'Perce Pearce',
   'Ben Sharpsteen'],
  'Written by': ['Ted Sears',
   'Richard Creedon',
   'Otto Englander',
   'Dick Rickard',
   'Earl Hurd',
   'Merrill De Maris',
   'Dorothy Ann Blank',
   'Webb Smith'],
  'Based on': ['Snow White', 'Brothers Grimm'],
  'Produced by': 'Walt Disney',
  'Starring': ['Adriana Caselotti',
   'Lucille La Verne',
   'Harry Stockwell',
   'Roy Atwell',
   'Pinto Colvig',
   'Otis Harlan',
   'Scotty Mattraw',
   'Billy Gilbert',
   'Eddie Collins',
   'Moroni Olsen',
   'Stuart Buchanan'],
  'Music by': ['Frank Churchill', 'Paul Smith', 'Leigh Harline'],
  'Production company': ['Walt Disney Productions'],
  'Distributed by': 'RKO Radio Pictures',
  'Release date': ['December 21, 1937 ( Carthay Circle Theatre , Los Angeles , CA , premiere)'],
  'Running time': '83 minutes',
  'Country': 'United St